<H1 style=text-align:center;><b>Projet Maths-Info : Traveling Salesman Problem</b></H1>
<h1 style=text-align:center;>(AMU) EADS 2024-2025</h1>

<h2 style=text-align:center;><i>Projet présenté par : Alexandre ROMANO / Victoria BOUCHET / Vahé TILDIAN</i></h2>

<br><br><br>

## <u>Introduction</u>

...


<br>

## <u>Fonctions utiles</u>

...

### distance.py

...

### generate.py

...

### quality.py

...

### visualisation.py

...

<br>

## <u>Implémentation des Heuristiques</u>

...

### Nearest Neighbor (neighbor.py)

...

### Cheapest Insertion (insertion.py)

...


### 2-opt (twoopt.py)

...

### Simulated Annealing (annealing.py)

...

<br>

## <u>Main</u>

...